## Accessing Sentinel-5P data on Azure

The [Sentinel-5P](https://sentinel.esa.int/web/sentinel/missions/sentinel-5p) mission provides daily global atmospheric measurements at a resolution of 3.5km x 7km on most bands.  This dataset represents the global archive of Sentinel-5P [Level 2](http://www.tropomi.eu/data-products/level-2-products) products, from 2018 to the present, in NetCDF format.  

This notebook demonstrates basic access to Sentinel-5 data on Azure.  Because Sentinel-1 data are in preview, the user needs to provide storage credentials.

This dataset is stored in the West Europe Azure region, so this notebook will run most efficiently on Azure compute located in the same region.  If you are using this data for environmental science applications, consider applying for an [AI for Earth grant](http://aka.ms/ai4egrants) to support your compute requirements.

This dataset is documented at [aka.ms/ai4edata-sentinel-5p](http://aka.ms/ai4edata-sentinel-5p).

Sentinel-5P data on Azure are maintained by [Sinergise](https://sinergise.com/).

### Environment setup

In [ ]:
import os
import fsspec
import xarray as xr
from azure.storage.blob import ContainerClient

# Not used directly, but needs to be installed to read NetCDF files with xarray
import h5netcdf

### Auth files

In [ ]:
# A plain-text file with a SAS token (starting with "?sv") on the first line
sas_file = os.path.expanduser('~/tokens/sentinel-5p_sas.txt')

### Constants

In [ ]:
product = 'L2__CH4___'
date = '2020/01/01'

### Azure storage constants

In [ ]:
lines = []
with open(sas_file,'r') as f:
    lines = f.readlines()
assert len(lines) >= 1
sas_token = lines[0].strip()
        
storage_account_name = 'sentinel5euwest'
container_name = 'sentinel-5p'
storage_account_url = 'https://' + storage_account_name + '.blob.core.windows.net/'

container_client = ContainerClient(account_url=storage_account_url, 
                                                 container_name=container_name,
                                                 credential=sas_token)

### List products matching our product/date

In [ ]:
prefix = '/'.join(['TROPOMI',product,date])
print(prefix)
generator = container_client.list_blobs(name_starts_with=prefix)
scene_paths = [blob.name for blob in generator]
print('Found {} matching scenes:\n'.format(len(scene_paths)))
for s in scene_paths:
    print(s.split('/')[-1])

### Print metadata for one scene

In [ ]:
# Choose the first scene for this product/date
scene_path = scene_paths[0]
url = storage_account_url + container_name + '/' + scene_path
print('Processing image at URL:\n{}'.format(url))

In [ ]:
import warnings; warnings.filterwarnings('ignore')
with fsspec.open(url+sas_token) as f:
    ds = xr.open_dataset(f)
print(ds)

In [ ]:
with fsspec.open(url+sas_token) as f:
    ds = xr.open_dataset(f,group='PRODUCT')
print(ds)